In [13]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [15]:
import os
import csv
import json
import time
import re
from curl_cffi import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup

# SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

from db_client import RDSClient

# ==========================================
# 환경 변수 및 RDSClient 설정
# ==========================================
rds = RDSClient()

# ==========================================
# DB 컬럼 및 초기 설정
# ==========================================
cols_result = rds.execute("SHOW COLUMNS FROM product_bottom")
if not cols_result:
    print("❌ 테이블 정보를 가져오지 못했습니다.")
    exit()

db_cols = [row['Field'] for row in cols_result]
placeholders = ", ".join([f":{col}" for col in db_cols])
insert_sql = f"INSERT IGNORE INTO product_bottom ({', '.join(db_cols)}) VALUES ({placeholders})"

# ==========================================
# CSV 로드
# ==========================================
goods_ids = []
try:
    with open("musinsa_bottom_ids.csv", newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader) # 헤더 건너뛰기
        for row in reader:
            if row: # 빈 줄 방지
                goods_ids.append(row[0])
except FileNotFoundError:
    print("❌ CSV 파일 없음")
    exit()

total_count = len(goods_ids)

# ==========================================
# 시작 위치 설정
# ==========================================
cnt_result = rds.execute("SELECT COUNT(*) as cnt FROM product_bottom")
saved_count = cnt_result[0]['cnt'] if cnt_result else 0
start_index = saved_count

print(f"전체: {total_count} / 저장됨: {saved_count} / 시작 인덱스: {start_index}")

# ==========================================
# 크롤링 시작 (curl_cffi + JSON Parsing)
# ==========================================
print("🚀 고속 크롤링 시작 (Next.js JSON 데이터 추출 방식)")

session = requests.Session(impersonate="chrome") 
session.headers.update({
    "Referer": "https://www.musinsa.com/",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

for idx in range(start_index, total_count):
    gid = goods_ids[idx]
    
    print(f"[{idx+1}/{total_count}] {gid} 요청 중...", end=" ")

    try:
        url = f"https://www.musinsa.com/products/{gid}"
        
        # 1. 요청
        response = session.get(url, timeout=10)
        
        if response.status_code != 200:
            print(f"⚠️ HTTP 실패: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        
        # 2. 봇 차단 감지
        page_title = soup.title.get_text(strip=True) if soup.title else ""
        if any(k in page_title for k in ["Access Denied", "Just a moment", "Security Check"]):
            print(f"\n🚨 [CRITICAL] 봇 차단 감지됨! ({gid}) - 종료합니다.")
            break

        # 3. Next.js 데이터 추출 (__NEXT_DATA__)
        next_data_tag = soup.find("script", {"id": "__NEXT_DATA__"})
        
        if not next_data_tag:
            print(f"⚠️ JSON 데이터 태그 없음")
            continue
            
        try:
            raw_json = json.loads(next_data_tag.string)
        except json.JSONDecodeError:
            print(f"⚠️ JSON 디코딩 실패")
            continue

        # ============================================================
        # 4. JSON 내부에서 상품 데이터 위치 찾기 (탐색 로직 개선)
        # ============================================================
        product_info = None
        page_props = raw_json.get("props", {}).get("pageProps", {})

        # [CASE 1] 분석하신 JSON 구조 (meta -> data)
        # 제공해주신 파일은 이 경로에 데이터가 있습니다.
        if "meta" in page_props and "data" in page_props["meta"]:
            candidate = page_props["meta"]["data"]
            # goodsNo가 일치하는지 확인 (문자열 변환 후 비교)
            if str(candidate.get("goodsNo", "")) == str(gid):
                product_info = candidate

        # [CASE 2] 기존 탐색 로직 (dehydratedState -> queries)
        # CASE 1에서 못 찾았을 경우 실행
        if not product_info:
            queries = page_props.get("dehydratedState", {}).get("queries", [])
            for query in queries:
                data_node = query.get("state", {}).get("data", {})
                
                if isinstance(data_node, dict):
                    if str(data_node.get("goodsNo", "")) == str(gid) or str(data_node.get("productNo", "")) == str(gid):
                        product_info = data_node
                        break
                    elif "product" in data_node and (str(data_node["product"].get("goodsNo", "")) == str(gid)):
                        product_info = data_node["product"]
                        break
                    elif "goods" in data_node and (str(data_node["goods"].get("goodsNo", "")) == str(gid)):
                        product_info = data_node["goods"]
                        break

        if not product_info:
            print(f"⚠️ JSON 내 상품 정보 매칭 실패")
            # 디버깅용 (필요시 사용)
            # with open(f"fail_{gid}.json", "w", encoding="utf-8") as f:
            #     json.dump(raw_json, f, ensure_ascii=False, indent=2)
            continue

        # ============================================================
        # 5. 데이터 매핑 (JSON 구조 차이 통합 처리)
        # ============================================================
        
        # 1) 상품명: goodsNm(새 구조) vs goodsName(구 구조) vs productName
        product_name = (product_info.get("goodsNm") or 
                        product_info.get("goodsName") or 
                        product_info.get("productName") or "")

        # 2) 브랜드: brandInfo 객체 내부 vs brandName 문자열
        brand_info = product_info.get("brandInfo", {})
        if isinstance(brand_info, dict) and "brandName" in brand_info:
            brand = brand_info["brandName"]
        else:
            brand = product_info.get("brandName", "") or product_info.get("brand", "")
        
        # 3) 가격 정보
        # 새 구조: goodsPrice 객체 사용 / 구 구조: price 객체 사용
        price_info = product_info.get("goodsPrice") or product_info.get("price") or {}
        normal_price = price_info.get("normalPrice") or price_info.get("originPrice") or 0
        sale_price = price_info.get("salePrice") or price_info.get("price") or 0
        discount = price_info.get("discountRate", 0)

        # 4) 카테고리
        # 새 구조: 'category' 객체 (단일) / 구 구조: 'categories' 리스트
        upper_category = ""
        lower_category = ""
        
        # CASE A: 단일 객체 (새 구조)
        cat_obj = product_info.get("category")
        if isinstance(cat_obj, dict):
            upper_category = cat_obj.get("categoryDepth1Title", "")
            lower_category = cat_obj.get("categoryDepth2Title", "")
        
        # CASE B: 리스트 (구 구조) - 데이터가 없으면 실행
        if not upper_category:
            cats = product_info.get("categories", [])
            if cats:
                upper_category = cats[0].get("depth1Title", "")
                lower_category = cats[0].get("depth2Title", "")

        # 5) 성별
        # 새 구조: "sex": ["남성", "여성"] 리스트 형태
        # 구 구조: "sex": "M" 문자열 형태
        sex_data = product_info.get("sex")
        gender = 0 # 기본값: 공용
        
        if isinstance(sex_data, list): # ["남성", "여성"] 형태
            if "남성" in sex_data and "여성" in sex_data: gender = 0
            elif "남성" in sex_data: gender = 1
            elif "여성" in sex_data: gender = 2
        else: # 문자열 코드 형태 (M, F)
            if sex_data in ["M", "MALE", "남성"]: gender = 1
            elif sex_data in ["F", "FEMALE", "여성"]: gender = 2

        # 6) 통계 (좋아요, 리뷰수, 별점)
        # 새 구조: goodsReview 객체 / 구 구조: goodsCount 또는 stat
        stat_info = (product_info.get("goodsReview") or 
                     product_info.get("goodsCount") or 
                     product_info.get("stat") or {})
        
        # 리뷰 수
        review_cnt = stat_info.get("totalCount") or stat_info.get("reviewCount") or 0
        
        # 별점 (satisfactionScore vs reviewAverage)
        rating = float(stat_info.get("satisfactionScore") or stat_info.get("reviewAverage") or 0.0)
        
        # 좋아요 수 (새 구조인 goodsReview에는 좋아요가 없는 경우가 많음. 없으면 0 처리)
        # goodsCount나 stat가 있다면 거기서 가져옴
        like_cnt = (product_info.get("goodsCount", {}).get("likeCount") or 
                    product_info.get("stat", {}).get("likeCount") or 0)

        # 7) 누적 판매
        cumulative = (product_info.get("cumulativeSales") or 
                      str(stat_info.get("purchaseCount", "")) or "")

        # 8) 스타일 태그
        tags_list = []
        
        try:
            # 1. 태그 API URL 생성
            tag_api_url = f"https://goods-detail.musinsa.com/api2/goods/{gid}/tags"
            
            # 2. API 요청 (기존 session 사용)
            # 별도 헤더 설정 없이 기존 세션의 헤더(User-Agent 등)를 그대로 사용합니다.
            tag_response = session.get(tag_api_url, timeout=5)
            
            if tag_response.status_code == 200:
                tag_json = tag_response.json()
                
                # 3. 데이터 추출 ("data" -> "tags" 리스트)
                # 제공해주신 JSON 구조: {"data": {"tags": [...]}}
                if "data" in tag_json and "tags" in tag_json["data"]:
                    tags_list = tag_json["data"]["tags"]
            else:
                print(f"⚠️ 태그 API 실패: {tag_response.status_code}")

        except Exception as e:
            print(f"⚠️ 태그 수집 중 에러: {e}")

        # 4. 리스트를 콤마로 구분된 문자열로 변환 (DB 저장용)
        # 예: ['반바지', '버뮤다'] -> "반바지,버뮤다"
        final_tags_str = ','.join(tags_list)

        # 수집 결과 확인용 출력
        if final_tags_str:
            print(f"✅ 태그: {final_tags_str}")
        else:
            print(f"⚠️ 태그 없음 (API 응답 비어있음)")


        # JSON 데이터 문자열 변환 (빈값 처리)
        size_json = "[]" # 사이즈 정보는 별도 로직이 없다면 빈 리스트
        
        # 핏/계절감 정보가 있다면 JSON 문자열로, 없으면 기본값
        # (product_info['goodsMaterial'] 등에서 추출 가능하나, 여기선 기본 구조 유지)
        fit_season_dict = {"핏": [], "계절감": []} 
        fit_json = json.dumps(fit_season_dict, ensure_ascii=False)

        # ============================================================
        # 8. DB 저장 (INSERT IGNORE 적용)
        # ============================================================
        
        # 파라미터 딕셔너리 생성
        params = {
            "product_id": gid,
            "product_name": product_name,
            "brand": brand,
            "original_price": normal_price,
            "sale_price": sale_price,
            "upper_category": upper_category,
            "lower_category": lower_category,
            "gender": gender,
            "rating": rating,
            "wish_count": like_cnt,  # wish_count 컬럼에 매핑
            "review_count": review_cnt,
            "size_info": size_json,
            "discount_rate": discount,
            "fit_season": fit_json,
            "cumulative_sales": cumulative,
            "style": final_tags_str  # ✅ 크롤링한 태그들이 여기에 들어갑니다
        }

        # INSERT IGNORE 구문 사용 (중복 에러 방지)
        insert_sql = f"INSERT IGNORE INTO product_bottom ({', '.join(db_cols)}) VALUES ({placeholders})"
        
        # 쿼리 실행
        rds.execute(insert_sql, params)
        print(f"✅ 저장 완료 ({product_name[:15]}...)")

    except Exception as e:
        print(f"❌ 처리 중 알 수 없는 에러 발생: {e}")
        # traceback.print_exc() # 디버깅 필요시 주석 해제

print("\n🎉 모든 작업 완료!")

DB HOST: musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com
DB USER: admin
DB PASSWORD: qkqajrwkrnrnrn9_
DB NAME: musinsa
DB PORT: 3306
✅ DB Engine (Pool) 생성 완료


KeyboardInterrupt: 